In [1]:
import re
from pathlib import PurePath
import pickle
from random import shuffle
import pandas as pd
import numpy as np

In [2]:
with open("clusters-by-entity-30.txt", "r") as g:
    pdb_clusters = g.readlines()
p = ['1a30']


In [32]:
pdb_clusters

['1YPZ_4 4LHU_4 1HXM_2 4LFH_2 6MWR_3 6D7G_3 4IML_2 1XF2_3 1XF3_2 1XF4_2 1P7K_2 2FR4_3 1I8M_4 5CP3_2 5CP7_2 7F2O_5 5Y9K_2 5UTZ_1 7LCN_2 6FXN_2 5Y9J_2 7VYR_1 5N2K_2 5N2K_4 5NJD_4 1NAK_2 5IT2_1 1IAI_2 3TT1_3 6LFW_1 6LFX_1 6LFV_1 4TVP_6 5UTY_3 6CH8_3 6MDT_4 6UTK_2 5UM8_6 6IEQ_6 5U7O_3 5T3S_6 5W6D_6 5V7J_6 6DE7_3 5U7M_3 5FYK_3 6CE0_4 5FYL_3 6V6W_2 6MCO_3 5FYJ_3 5UTF_6 5WDU_7 5CEZ_6 6CH7_3 6CH9_3 6OPA_4 4TOY_2 7KMD_2 6PZY_2 1JNN_2 1JNL_2 6O2A_1 6O2B_1 6O2C_1 6O28_1 6O29_1 4K2U_2 6IUT_1 7C01_3 7L77_2 7LL2_4 4PS4_1 6XPQ_2 5EPM_1 4OKV_1 4QT5_2 6PZH_2 1XGY_2 7L7E_3 6MTY_1 7BQ5_2 4CAD_2 6A0X_1 6A0Z_2 1Q0Y_2 1Q0X_2 4QCI_2 7EY5_4 3UTZ_2 3UZE_1 3UZQ_1 1DZB_1 3UZV_2 5AAM_1 5AAW_1 7DET_2 3UYP_1 2G60_2 6XKR_1 5CZX_2 6H06_1 6H0E_1 5V7U_2 5TKK_2 6AOD_2 5TKK_3 3PP3_1 3PP4_1 6Y9A_2 3OPZ_2 7M6D_1 6X97_4 6X96_4 6X98_3 4JO1_1 4JO2_1 4JO3_1 4JO4_1 5V6L_1 6PEH_2 6P65_3 5M63_2 6X1S_2 6X1T_2 6X1W_2 6I9I_2 7O9S_2 6CEZ_1 5OB5_2 5NUZ_1 6IDG_2 6IDH_2 6KDH_2 6KDI_2 3VW3_2 1EHL_3 1KEG_3 3CMO_2 3BSZ_4 3WFB_2 3WFC_2 3WFD

In [89]:
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = [value]

In [90]:
def read_strip_and_return_clean_list(file):
    if file == None:
        return []
    else:
        cleaned_list = []
        with open(file) as b:
            for raw_line in b.readlines():
                cleaned_list.append(raw_line.strip())  
        return cleaned_list    

In [91]:
pdbs = read_strip_and_return_clean_list('pdbbind_2020_general_cleaned_ids_wout_undocked.txt')
print(len(pdbs))

14780


In [116]:
def create_train_val_split_dict(pdbs):
    train_and_val_clusters = {}
    for p in pdbs:
        found = False
        for cluster in pdb_clusters:
            if not found:
                if p.upper() in cluster:
                    found = True
                    append_value(train_and_val_clusters, pdb_clusters.index(cluster), p)
    train_list = []
    val_list = []
    shuffled_keys = list(train_and_val_clusters.keys())
    shuffle(shuffled_keys)
    for i in shuffled_keys:
        cluster = train_and_val_clusters[i]
        if isinstance(cluster, str):
            cluster = [cluster]
        if len(cluster) + len(train_list) < len(pdbs)*0.8:
            train_list.extend(cluster)
        else:
            val_list.extend(cluster)
    return train_list, val_list



In [112]:
def generate_types_str_regression(
    index_file,
    include_ids_file,
    mode=False,
    crossdock_dict_file=None,
    file_string=None
    ):
    """Generate a portion of a types file."""

    def types_line_regression(receptor_pdb, ligand_sdf, affinity, metric):
        affinities = [-1, -1, -1]
        affinities[['Ki', 'Kd', 'IC50'].index(metric)] = affinity
        affinity_str = '{0} {1} {2}'.format(*affinities)
        return '{0} {1} {2}\n'.format(affinity_str, receptor_pdb, ligand_sdf)
#     directory = expand_path(directory)
    
    def read_strip_and_return_clean_list(file):
        if file == None:
            return []
        else:
            cleaned_list = []
            with open(file) as b:
                for raw_line in b.readlines():
                    cleaned_list.append(raw_line.strip())  
            return cleaned_list    
    train_str = ''
    val_str = ''
    actual_pdbs = read_strip_and_return_clean_list(include_ids_file)
    train_list, val_list = create_train_val_split_dict(actual_pdbs)
    
    with open(index_file) as f:
        index_file_lines = f.readlines()[6:]
        shuffle(index_file_lines)
        for line in index_file_lines:
            pdb = line.split()[0]
            if pdb in read_strip_and_return_clean_list(include_ids_file):
                types_str = ''
                protein_file = f'{pdb}/{pdb}_protein_cleaned.parquet'
                if not mode: 
                    ligand_file = f'{pdb}/{pdb}_ligand_pymol.parquet'
                elif mode == 'redocking':
                    ligand_file = f'{pdb}/{pdb}_redocking_best_pose.parquet'
                elif mode == 'crossdocking':
                    cross_dock_dict = pickle.load(open(crossdock_dict_file, 'rb'))
                    ligand_file = f'{pdb}/{pdb}_crossdocking_{file_string}_similiar_best_pose.parquet' 
                    protein_file = f'{cross_dock_dict[pdb]}/{cross_dock_dict[pdb]}_protein_cleaned.parquet'
                pdbid, affinity, metric = None, None, None
                affinity = line.split()[3]
                metric = (re.split(r'~|=|>|<', line.split()[4]))[0]
                types_line = types_line_regression(
                protein_file, ligand_file, affinity, metric)
                types_str += types_line
                if pdb in train_list:
                    train_str += types_str
                else:
                    val_str += types_str
        return train_str[:-1], val_str[:-1]

In [113]:
def read_strip_and_return_clean_list(file):
    if file == None:
        return []
    else:
        cleaned_list = []
        with open(file) as b:
            for raw_line in b.readlines():
                cleaned_list.append(raw_line.strip())  
        return cleaned_list  

def create_size_of_dataset_control(file_for_size, file_to_sample_from, output_filename):
    list_for_size = read_strip_and_return_clean_list(file_for_size)
    list_to_sample_from = read_strip_and_return_clean_list(file_to_sample_from)
    shuffle(list_to_sample_from)
    new_list = list_to_sample_from[:len(list_for_size)]
    pdb_list_to_txt_file(output_filename, new_list)
    return None

def pdb_list_to_txt_file(file_name, pdb_list): 
    with open(file_name, 'w') as f:
        for p in pdb_list:
            f.write(p + '\n')

In [105]:
# CREATE SIZE CONTROL FOR DEBIASED TRAINING
create_size_of_dataset_control('pdbbind_2020_general_cleaned_ids_wout_undocked_both_threshold_30.txt', 
                              'pdbbind_2020_general_cleaned_ids_wout_undocked.txt',
                              'pdbbind_2020_general_cleaned_ids_wout_undocked_both_threshold_30_size_control.txt')

In [117]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020', 'pdbbind_2020_general_cleaned_ids_wout_undocked.txt')
with open('pdbbind_2020_general_pymol_train.types', 'w') as f:
    f.write(new_types_file[0])
with open('pdbbind_2020_general_pymol_val.types', 'w') as f:
    f.write(new_types_file[1])

In [118]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020', 'pdbbind_2020_general_cleaned_ids_wout_undocked_sequence_threshold_100.txt')
with open('pdbbind_2020_general_pymol_sequence_threshold_100_train.types', 'w') as f:
    f.write(new_types_file[0])
with open('pdbbind_2020_general_pymol_sequence_threshold_100_val.types', 'w') as f:
    f.write(new_types_file[1])

In [119]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020', 'pdbbind_2020_general_cleaned_ids_wout_undocked_sequence_threshold_90.txt')
with open('pdbbind_2020_general_pymol_sequence_threshold_90_train.types', 'w') as f:
    f.write(new_types_file[0])
with open('pdbbind_2020_general_pymol_sequence_threshold_90_val.types', 'w') as f:
    f.write(new_types_file[1])

In [120]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020', 'pdbbind_2020_general_cleaned_ids_wout_undocked_sequence_threshold_30.txt')
with open('pdbbind_2020_general_pymol_sequence_threshold_30_train.types', 'w') as f:
    f.write(new_types_file[0])
with open('pdbbind_2020_general_pymol_sequence_threshold_30_val.types', 'w') as f:
    f.write(new_types_file[1])

In [121]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020', 'pdbbind_2020_general_cleaned_ids_wout_undocked_tanimoto_threshold_100.txt')
with open('pdbbind_2020_general_pymol_tanimoto_threshold_100_train.types', 'w') as f:
    f.write(new_types_file[0])
with open('pdbbind_2020_general_pymol_tanimoto_threshold_100_val.types', 'w') as f:
    f.write(new_types_file[1])

In [122]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020', 'pdbbind_2020_general_cleaned_ids_wout_undocked_tanimoto_threshold_90.txt')
with open('pdbbind_2020_general_pymol_tanimoto_threshold_90_train.types', 'w') as f:
    f.write(new_types_file[0])
with open('pdbbind_2020_general_pymol_tanimoto_threshold_90_val.types', 'w') as f:
    f.write(new_types_file[1])

In [123]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020', 'pdbbind_2020_general_cleaned_ids_wout_undocked_tanimoto_threshold_30.txt')
with open('pdbbind_2020_general_pymol_tanimoto_threshold_30_train.types', 'w') as f:
    f.write(new_types_file[0])
with open('pdbbind_2020_general_pymol_tanimoto_threshold_30_val.types', 'w') as f:
    f.write(new_types_file[1])

In [124]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020', 'pdbbind_2020_general_cleaned_ids_wout_undocked_both_threshold_100.txt')
with open('pdbbind_2020_general_pymol_both_threshold_100_train.types', 'w') as f:
    f.write(new_types_file[0])
with open('pdbbind_2020_general_pymol_both_threshold_100_val.types', 'w') as f:
    f.write(new_types_file[1])

In [125]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020', 'pdbbind_2020_general_cleaned_ids_wout_undocked_both_threshold_90.txt')
with open('pdbbind_2020_general_pymol_both_threshold_90_train.types', 'w') as f:
    f.write(new_types_file[0])
with open('pdbbind_2020_general_pymol_both_threshold_90_val.types', 'w') as f:
    f.write(new_types_file[1])

In [126]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020', 'pdbbind_2020_general_cleaned_ids_wout_undocked_both_threshold_30.txt')
with open('pdbbind_2020_general_pymol_both_threshold_30_train.types', 'w') as f:
    f.write(new_types_file[0])
with open('pdbbind_2020_general_pymol_both_threshold_30_val.types', 'w') as f:
    f.write(new_types_file[1])

In [127]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020', 'pdbbind_2020_general_cleaned_ids_wout_undocked_both_threshold_30_size_control.txt')
with open('pdbbind_2020_general_pymol_both_threshold_30_size_control_train.types', 'w') as f:
    f.write(new_types_file[0])
with open('pdbbind_2020_general_pymol_both_threshold_30_size_control_val.types', 'w') as f:
    f.write(new_types_file[1])

In [16]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020','casf_2016_ids.txt')
with open('casf_2016_pymol.types', 'w') as f:
    f.write(new_types_file)

# For Redocked 

In [128]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020', 'pdbbind_2020_general_cleaned_ids_wout_undocked.txt', mode = 'redocking')
with open('pdbbind_2020_general_pymol_redocked_train.types', 'w') as f:
    f.write(new_types_file[0])
with open('pdbbind_2020_general_pymol_redocked_val.types', 'w') as f:
    f.write(new_types_file[1])

In [129]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020', 'pdbbind_2020_general_cleaned_ids_wout_undocked_sequence_threshold_100.txt', mode = 'redocking')
with open('pdbbind_2020_general_pymol_redocked_sequence_threshold_100_train.types', 'w') as f:
    f.write(new_types_file[0])
with open('pdbbind_2020_general_pymol_redocked_sequence_threshold_100_val.types', 'w') as f:
    f.write(new_types_file[1])

In [130]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020', 'pdbbind_2020_general_cleaned_ids_wout_undocked_sequence_threshold_90.txt', mode = 'redocking')
with open('pdbbind_2020_general_pymol_redocked_sequence_threshold_90_train.types', 'w') as f:
    f.write(new_types_file[0])
with open('pdbbind_2020_general_pymol_redocked_sequence_threshold_90_val.types', 'w') as f:
    f.write(new_types_file[1])

In [131]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020', 'pdbbind_2020_general_cleaned_ids_wout_undocked_sequence_threshold_30.txt', mode = 'redocking')
with open('pdbbind_2020_general_pymol_redocked_sequence_threshold_30_train.types', 'w') as f:
    f.write(new_types_file[0])
with open('pdbbind_2020_general_pymol_redocked_sequence_threshold_30_val.types', 'w') as f:
    f.write(new_types_file[1])

In [132]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020', 'pdbbind_2020_general_cleaned_ids_wout_undocked_tanimoto_threshold_100.txt', mode = 'redocking')
with open('pdbbind_2020_general_pymol_redocked_tanimoto_threshold_100_train.types', 'w') as f:
    f.write(new_types_file[0])
with open('pdbbind_2020_general_pymol_redocked_tanimoto_threshold_100_val.types', 'w') as f:
    f.write(new_types_file[1])

In [133]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020', 'pdbbind_2020_general_cleaned_ids_wout_undocked_tanimoto_threshold_90.txt', mode = 'redocking')
with open('pdbbind_2020_general_pymol_redocked_tanimoto_threshold_90_train.types', 'w') as f:
    f.write(new_types_file[0])
with open('pdbbind_2020_general_pymol_redocked_tanimoto_threshold_90_val.types', 'w') as f:
    f.write(new_types_file[1])

In [134]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020', 'pdbbind_2020_general_cleaned_ids_wout_undocked_tanimoto_threshold_30.txt', mode = 'redocking')
with open('pdbbind_2020_general_pymol_redocked_tanimoto_threshold_30_train.types', 'w') as f:
    f.write(new_types_file[0])
with open('pdbbind_2020_general_pymol_redocked_tanimoto_threshold_30_val.types', 'w') as f:
    f.write(new_types_file[1])

In [135]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020', 'pdbbind_2020_general_cleaned_ids_wout_undocked_both_threshold_100.txt', mode = 'redocking')
with open('pdbbind_2020_general_pymol_redocked_both_threshold_100_train.types', 'w') as f:
    f.write(new_types_file[0])
with open('pdbbind_2020_general_pymol_redocked_both_threshold_100_val.types', 'w') as f:
    f.write(new_types_file[1])

In [136]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020', 'pdbbind_2020_general_cleaned_ids_wout_undocked_both_threshold_90.txt', mode = 'redocking')
with open('pdbbind_2020_general_pymol_redocked_both_threshold_90_train.types', 'w') as f:
    f.write(new_types_file[0])
with open('pdbbind_2020_general_pymol_redocked_both_threshold_90_val.types', 'w') as f:
    f.write(new_types_file[1])

In [137]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020', 'pdbbind_2020_general_cleaned_ids_wout_undocked_both_threshold_30.txt', mode = 'redocking')
with open('pdbbind_2020_general_pymol_redocked_both_threshold_30_train.types', 'w') as f:
    f.write(new_types_file[0])
with open('pdbbind_2020_general_pymol_redocked_both_threshold_30_val.types', 'w') as f:
    f.write(new_types_file[1])

In [138]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020', 'pdbbind_2020_general_cleaned_ids_wout_undocked_both_threshold_30_size_control.txt', mode= 'redocking')
with open('pdbbind_2020_general_pymol_redocked_both_threshold_30_size_control_train.types', 'w') as f:
    f.write(new_types_file[0])
with open('pdbbind_2020_general_pymol_redocked_both_threshold_30_size_control_val.types', 'w') as f:
    f.write(new_types_file[1])

In [27]:
new_types_file = generate_types_str_regression('../INDEX_general_PL_data.2020', 'casf_2016_ids.txt', mode = 'redocking')
with open('casf_2016_pymol_redocked.types', 'w') as f:
    f.write(new_types_file)

# For crossdocking

In [17]:
new_types_file = generate_types_str_regression(
    '../INDEX_general_PL_data.2020', 
    None, 
    'casf_2016_ids.txt', 
    mode = 'crossdocking', 
    crossdock_dict_file = 'casf_most_similiar_crossdock_dict',
    file_string = 'most_structure')
with open('casf_2016_pymol_crossdocked_most_similiar_structure.types', 'w') as f:
    f.write(new_types_file)

In [18]:
new_types_file = generate_types_str_regression(
    '../INDEX_general_PL_data.2020', 
    None, 
    'casf_2016_ids.txt', 
    mode = 'crossdocking', 
    crossdock_dict_file = 'casf_least_similiar_crossdock_dict',
    file_string = 'least_structure')
with open('casf_2016_pymol_crossdocked_least_similiar_structure.types', 'w') as f:
    f.write(new_types_file)

In [19]:
new_types_file = generate_types_str_regression(
    '../INDEX_general_PL_data.2020', 
    None, 
    'casf_2016_ids.txt', 
    mode = 'crossdocking', 
    crossdock_dict_file = 'casf_most_similiar_ligand_crossdock_dict',
    file_string = 'most_ligand')
with open('casf_2016_pymol_crossdocked_most_similiar_ligand.types', 'w') as f:
    f.write(new_types_file)

In [20]:
new_types_file = generate_types_str_regression(
    '../INDEX_general_PL_data.2020', 
    None, 
    'casf_2016_ids.txt', 
    mode = 'crossdocking', 
    crossdock_dict_file = 'casf_least_similiar_ligand_crossdock_dict',
    file_string = 'most_ligand')
with open('casf_2016_pymol_crossdocked_least_similiar_ligand.types', 'w') as f:
    f.write(new_types_file)